### Portail meteo.data.gouv.fr - Téléchargement-affichage-Extraction des données MENSUELLES LATEST de Météo-France (Métropole & outre-mer)
Une connexion internet est nécessaire pour l'accès en ligne aux archives des données
1) Téléchargement des fichiers et décompression automatique, pour plusieurs départements si besoin
2) Tracé du graphique chronologique pour le paramètre Précipitations RR des postes choisis par l'utilisateur
3) Sauvegarde des données intégrales et du graphique dans un fichier Excel (tous paramètres de la période "Latest" pour tous les postes des départements concernés).

data : https://meteo.data.gouv.fr/ (6 min, horaire, quotidien, mensuel)<br>
Fiche PDF des postes : https://www.data.gouv.fr/fr/datasets/r/bee4b0c7-260a-40fe-b463-ed5631d6dc39 (paramètres et périodes de mesure)<br>
Fichier CSV descriptif champs: https://www.data.gouv.fr/fr/datasets/r/6d4ac560-8f7c-477f-9a3f-3c33137fc04e

Utilisez mon autre script pour visualiser sous forme de carte la liste des postes météorologiques fournie par Météo-France sous forme de fichier JSON https://meteo.data.gouv.fr/https://www.data.gouv.fr/fr/datasets/r/1fe544d8-4615-4642-a307-5956a7d90922

NB:
- Les données LATEST correspondent aux DERNIERS fichiers mis à jour quotidiennement, et qui vont du mois de janvier de l'année précédente au mois en cours même partiel.
- Les données MENSUELLES ne sont pas simplement équivalentes à des données QUOTIDIENNE agrégées (les paramètres ne sont pas strictement les mêmes. Par exemple, il existe des des NOMBRES DE JOURS DE PLUIE/GELEE/CHALEUR/etc.., ainsi que des précipitations MENSUELLES ESTIMEES 'RR_ME' plus anciennes que les mesures)

Auteur: https://github.com/loicduffar

##### 1) Télécharge les archives ZIP & décompresse les fichiers CSV LATEST (DERNIERE PERIODE DISPONIBLE)

- Définir les chemins d'enregistrement pour les archives GZ et pour les fichier CSV décompressés
- Définir les postes à interroger par leur code (voir fichier "fiches.json" sur https://www.data.gouv.fr/fr/datasets/r/1fe544d8-4615-4642-a307-5956a7d90922)
- Les archives LATEST sont automatiquement téléchargées-décompressées, et les fichiers CSV sont lus



In [26]:
############ Auteur: L. Duffar ###########
############ Décembre 2023 ###########
# python 3.8.12
# Télécharge les les archives ZIP & et décompresse les fichiers CSV LATEST (DERNIERE PERIODE DISPONIBLE)
import os
import requests
import pandas as pd
import numpy as np
import datetime
import time
import sys
import gzip

# ================ Personalisation ====================
# Chemin d'enregistrement des archives gz et des fichiers CSV décompressés
folder_gz= r"X:\1-COMMUN\DIS\Documentation\Hydrologie\Documentation externe\Climat France\Météo-France\meteo.data\archives\2023 Déc\base\M"
folder_csv= r"X:\1-COMMUN\DIS\Documentation\Hydrologie\Documentation externe\Climat France\Météo-France\meteo.data\base\M"

# Numéro des Postes météo souhaités (chaine de 8 caractères) - voir fichier "fiches.json" sur https://www.data.gouv.fr/fr/datasets/r/1fe544d8-4615-4642-a307-5956a7d90922
# LES DEPARTEMENTS CORRESPONDANTS DOIVENT ËTRE PRESENTS DANS LA LISTE DES URLS (voir ======= Initialisation ==========)
postes= [
        '04088001', '04039001', '04230001',                  # Forcalquier, Castellane, Valensole \
        # '05046001',                                           # Embrun \
        # '06088001', '06029001',                               # Nice, Cannes \
        '13001009', '13111002', '13103001','13055001',        # Aix en Provence, Vauvenargue, Salon, Marseille, \
        '83031001', '83061001', '83137001',                   # Le Luc, Fréjus, Toulon \
        '84003002', '84009002'                                # Apt-Viton, Bastide des Jourdans \
        ]                               

# ================ Extrait du fichier data CSV ============= (les colonnes sont en réalité séparées par des points-virgules)
# NUM_POSTE	NOM_USUEL	LAT	LON	ALTI	AAAAMM	RR	QRR	NBRR	RR_ME	RRAB	QRRAB	RRABDAT	NBJRR1	NBJRR5	NBJRR10	NBJRR30
# 13001009	AIX EN PROVENCE	43.5295	5.4245	173	202201	3.6	1	31		1.4	1	4	1	0	0	0
# 13001009	AIX EN PROVENCE	43.5295	5.4245	173	202202	26.1	28		24.7	1	14	2	1	1	0
# 13001009	AIX EN PROVENCE	43.5295	5.4245	173	202203	6.6	1	31		3.6	1	30	2	0	0	0
# 13001009	AIX EN PROVENCE	43.5295	5.4245	173	202204	27	1	30		17.6	1	23	4	2	1	0
# 13001009	AIX EN PROVENCE	43.5295	5.4245	173	202205	26	1	31		16.8	1	8	3	2	1	0

# ================ Extrait du fichier CSV Descriptif de quelques paramètres (les plus souvent utiles, précipitations et températures) =============(les colonnes sont en réalité séparées par ":")
# NUM_POSTE       	 numéro Météo-France du poste sur 8 chiffres
# NOM_USUEL       	 nom usuel du poste
# LAT             	 latitude, négative au sud (en degrés et millionièmes de degré)
# LON             	 longitude, négative à l’ouest de GREENWICH (en degrés et millionièmes de degré)
# ALTI            	 altitude du pied de l'abri ou du pluviomètre si pas d'abri (en m)
# AAAAMM          	 mois
# RR              	 cumul mensuel des hauteurs de précipitation (en mm et 1/10)
# QRR             	 code qualité de RR
# NBRR            	 nombre de valeurs présentes de hauteur de précipitation quotidienne
# RR_ME           	 cumul mensuel estimé des hauteurs de précipitation (en mm et 1/10)
# RRAB            	 précipitation maximale tombée en 24 heures au cours du mois
# QRRAB           	 code qualite de RRAB
# NBJRR1          	 nombre de jours avec RR ≥ 1.0 mm
# TX              	 moyenne mensuelle des températures maximales (TX) quotidiennes (en °C et 1/10)
# QTX             	 code qualité de TX
# TN              	 moyenne mensuelle des températures minimales (TN) quotidiennes (en °C et 1/10)
# QTN             	 code qualité de TN
# TM              	 moyenne mensuelle des (TN+TX)/2 quotidiennes (en °C et 1/10)
# QTM             	 code qualité de TM
# TMM             	 moyenne mensuelle des températures moyennes (TM) quotidiennes (en °C et 1/10)
# QTMM            	 code qualité du TMM
# TMMIN           	 minimum mensuel des moyennes (TN+TX)/2 quotidiennes (en °C et 1/10)
# QTMMIN          	 code qualité de TMMIN
# TMMAX           	 maximum mensuel des moyennes (TN+TX)/2 quotidiennes (en °C et 1/10) 
# QTMMAX          	 code qualité du TMMAX
# NBJGELEE        	 nombre de jours avec gelée

# ================ Initialisation ====================
now= datetime.datetime.now()
print(now.strftime("%Y-%m-%d %H:%M"))
# Structure du nom des fichiers de données mensuelles (1 fichier par département dont le numéro sera ajouté automatiquement au début et à la fin du template ci-dessous)
template_start= 'MENSQ_'
# Fin du nom dépendant de l'année en cours
template_end= '_latest-' + str(now.year-1) + '-' + str(now.year) + '.csv'

# ---------------- urls de téléchargement des archives des dernières données depuis janvier de l'année précédente https://meteo.data.gouv.fr/ (Ajouter d'autres départements si besoin)
url_liste_postes= "https://www.data.gouv.fr/fr/datasets/r/1fe544d8-4615-4642-a307-5956a7d90922"
url_desc_m = "https://www.data.gouv.fr/fr/datasets/r/6d4ac560-8f7c-477f-9a3f-3c33137fc04e"
urls= dict()
urls['04']= "https://www.data.gouv.fr/fr/datasets/r/5c2d6f73-4162-4f36-bb41-98d7bc3e2ee1" 
urls['05']= 'https://www.data.gouv.fr/fr/datasets/r/589b53fd-adef-44a4-b306-1dee1500d2d2'
urls['06']= 'https://www.data.gouv.fr/fr/datasets/r/f8bdc686-8c37-4ced-8c36-407148bca194'
urls['13']= 'https://www.data.gouv.fr/fr/datasets/r/78fe3a8a-7f32-44dd-a1b5-8acb841d6cc1'
urls['83']= 'https://www.data.gouv.fr/fr/datasets/r/dc5ee912-e84a-4f15-bd08-7f326795ce41'
urls['84']= 'https://www.data.gouv.fr/fr/datasets/r/75526d6c-81d5-4aad-8961-e9fc2a43f7b5'
# print(urls)

# ---------------- Définition des fonctions
def convert_to_date(aaaamm):
    return pd.to_datetime(str(aaaamm), format='%Y%m', errors='coerce')

def download_file(url, filename):
    file= os.path.join(folder_gz, filename) + '.gz'
    print('Téléchargement: ', file)
    response = requests.get(url)
    if response.status_code == 200:
        with open(file, 'wb') as f:
            f.write(response.content)
    else:
        print("Fichier d'archive non présent à l'url habituelle: ", file)

def decompress_gz(filename):
    file= os.path.join(folder_gz, filename) + '.gz'
    
    if os.path.exists(file):
        with gzip.open(file, 'rb') as f_in:
            file= os.path.join(folder_csv, filename) 
            print('Décompression', file)
            with open(file, 'wb') as f_out:
                f_out.write(f_in.read())
    else:
        print("Fichier d'archive non trouvé: ", file)

def read_csv(filename):
    file= os.path.join(folder_csv, filename)
    if os.path.exists(file):
        print('Lecture: ', file)
        df= pd.read_csv(file, header=0, sep=";", dtype={"NUM_POSTE":str, 'AAAAMM':str}, parse_dates=['AAAAMM'], date_parser= convert_to_date)
    else:
        print("Fichier CSV non trouvé: ", file)
    return df

# ================ Traitement ====================
# Lit en ligne le fichier "MENSQ_descriptif_champs.csv" de description des champs et le stocke dans un dataframe pandas
# définit le dataframe pandas avec les 2 colonnes "param" et "name_long" pour la description des champs 
df_desc_m = pd.read_csv(url_desc_m, sep=":", header= None, index_col=0, names= ["param", "name_long", 'complement'], dtype={"param":str, "name_long":str, 'complement':str}, encoding= 'utf-8')
df_desc_m.index= df_desc_m.index.str.strip()
df_desc_m['name_long']= df_desc_m['name_long'].str.strip()

# fait la liste des départements concernés et garde les 2 premiers chiffres uniques
departements = []
for poste in postes:
    departements.append(poste[:2])
# On ne garde que départements uniques
departements = list(set(departements)) # set() ignore automatiquement les doublons (ce qui évite de passer par array numpy pour utiliser la fonction np.unique() )
# On trie la liste
departements.sort()
# On affiche la liste
print('Départements concernés: ', departements)

# Téléchargement/décompression/lecture des fichiers dans une boucle sur les départements (urls tirées du dictionnaire 'urls')
for i, departement in enumerate(departements):
    # On récupère l'url
    url = urls[departement]
    # Formation du nom du fichier à partir du template et du numéro de département
    filename = f"{template_start}{departement}{template_end}"
    # download_file(url, filename)
    # decompress_gz(filename)
    # Lecture du fichier CSV dans un dataframe pandas
    df_departement= read_csv(filename)
    if i== 0:
        # initialisation du dataframe final df avec les données du premier département        
        df= df_departement
    else:
        df= pd.concat([df, df_departement])        

# supprime les espaces avant et après le numéro de poste
df['NUM_POSTE'] = df['NUM_POSTE'].str.strip()
# renome la colonne "AAAMM" en "DATE"
df.rename(columns={'AAAAMM':'DATE'}, inplace=True)
# supprime les espaces dans les noms de colonnes
df.columns= df.columns.str.strip()
display(df)

2023-12-25 19:51
Départements concernés:  ['04', '13', '83', '84']
Lecture:  X:\1-COMMUN\DIS\Documentation\Hydrologie\Documentation externe\Climat France\Météo-France\meteo.data\base\M\MENSQ_04_latest-2022-2023.csv
Lecture:  X:\1-COMMUN\DIS\Documentation\Hydrologie\Documentation externe\Climat France\Météo-France\meteo.data\base\M\MENSQ_13_latest-2022-2023.csv
Lecture:  X:\1-COMMUN\DIS\Documentation\Hydrologie\Documentation externe\Climat France\Météo-France\meteo.data\base\M\MENSQ_83_latest-2022-2023.csv
Lecture:  X:\1-COMMUN\DIS\Documentation\Hydrologie\Documentation externe\Climat France\Météo-France\meteo.data\base\M\MENSQ_84_latest-2022-2023.csv


,NUM_POSTE,NOM_USUEL,LAT,LON,ALTI,DATE,RR,QRR,NBRR,RR_ME,...,QNEIGETOTM,NEIGETOTAB,QNEIGETOTAB,NEIGETOTABDAT,NBJNEIGETOT1,NBJNEIGETOT10,NBJNEIGETOT30,NBJGREL,NBJORAG,NBJBROU
0,04006005,ALLOS_SAPC,44.242500,6.625333,1400,2022-01-01,4.2,1.0,31,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,04006005,ALLOS_SAPC,44.242500,6.625333,1400,2022-02-01,29.4,1.0,28,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,04006005,ALLOS_SAPC,44.242500,6.625333,1400,2022-03-01,13.0,1.0,31,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,04006005,ALLOS_SAPC,44.242500,6.625333,1400,2022-04-01,53.6,1.0,30,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,04006005,ALLOS_SAPC,44.242500,6.625333,1400,2022-05-01,38.0,1.0,31,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427,84150001,VISAN,44.336667,4.905500,141,2023-08-01,49.0,1.0,31,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
428,84150001,VISAN,44.336667,4.905500,141,2023-09-01,56.9,1.0,30,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
429,84150001,VISAN,44.336667,4.905500,141,2023-10-01,131.1,1.0,31,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
430,84150001,VISAN,44.336667,4.905500,141,2023-11-01,44.6,1.0,30,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##### 2) Tracé du graphique des précipitations mensuelles

- Définir les dates de début et de fin du graphique comme suit:
    - None pour prendre le début et la fin du fichier LATEST (premier jour de l'année précédente et veille du téléchargement)
    - ou définir la date de début et de fin
- Définir le ou les paramètres à extraire (au minimum les précipitations RR car le graphique est uniquement prévu pour cela)

In [27]:
# trace un graphique plotly de df avec la colonne 'DATE' en abscisse et la colonne 'RR' en ordonnée
# chaque poste de la colonnee 'NUM_POSTE' est représenté dans un subplot séparé
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly

#-------------- Personalisation du graphique 
# Paramètres à représenter: 'RR' obligatoire en premier, pour le graphique dont la présentation actuelle est spécialement adaptée aux précipitations
param_list= ['RR', 'TM', 'FXYAB']

# Période à représenter (None pour le début ou la fin du fichier par défaut)
start_date= None
# start_date= datetime.datetime(2022, 5, 1)

end_date= None
# end_date= datetime.datetime(now.year, now.month, now.day)

#-------------- initialisation des paramètres du graphique
# Période par défaut if start_date= None ou  end_date= None
if start_date is None:
    start_date= df['DATE'].min()
if end_date is None:
    end_date= df['DATE'].max()

# filtre le dataframe sur les postes choisis
df_postes= df[df['NUM_POSTE'].isin(postes)]
# filtre le dataframe sur la période souhaitée
df_postes= df_postes[(df_postes['DATE'] >= start_date) & (df_postes['DATE'] <= end_date)]

#  Couleurs des courbes
color_sequence_bar = plotly.colors.qualitative.Set3
color_sequence_scatter = plotly.colors.qualitative.Pastel

# Axe des ordonnées secondaire (cumulé)
specs= [[{'secondary_y': True}]]*len(postes)

#-------------- Graphique
fig = make_subplots(rows= len(postes), cols= 1,# shared_xaxes= True, # shared_xaxes= True inutile grâce à  matches='x' dans update_xaxes qui fait la même chose sans supprimer la graduation des dates des subplots 
                    subplot_titles= [f"{poste} - {df_postes[df_postes['NUM_POSTE'] == poste]['NOM_USUEL'].iloc[0]}" for poste in postes], specs= specs)

for i, poste in enumerate(postes):
    df_poste = df_postes[df_postes['NUM_POSTE'] == poste]
    fig.add_trace(go.Bar(x= df_poste['DATE'], y= df_poste['RR'], 
                          # name= poste
                         name= df_postes[df_postes['NUM_POSTE'] == poste]['NOM_USUEL'].iloc[0],
                         marker= dict(color=color_sequence_bar[i % len(color_sequence_bar)]),  # Utilisez la couleur correspondante de la séquence
                         ),
                row= i+1, col= 1)
    fig.add_trace(go.Scatter(x= df_poste['DATE'], y= df_poste['RR'].cumsum(), 
                             name= f"Cumulé {poste}", yaxis= "y2",
                             line= dict(color= 'gray'),
                            #  line= dict(color= color_sequence_scatter[i % len(color_sequence_scatter)]), # sinon l'incrémentation dans la séquence de couleurs est de +2 à cause de la trace Bar
                             mode='lines' 
                             ), 
                 row=i+1, col=1, secondary_y= True)

fig.update_layout(title_text= 'meteo.data.gouv.fr - ' + "Précipitations mensuelles (mm) - " + now.strftime("%Y-%m-%d") , title_x= 0.5, 
                  height= 133.*len(postes), width= 1000,
                  hovermode= 'x unified', hoverlabel= dict(bgcolor='rgba(255,255,255,0.6)'),
                  )

# Survol par la souris : ligne verticale à travers tous les subplots matérialisant l'abscisse 
xlast= 'x' + str(len(postes))
# fig.update_traces(xaxis= xlast)  # xlast est le nom de l'axe des abscisses du dernier subplot

# Définition générale des axes
max_value= df_postes['RR'].max()
max_value_cum= df_postes.groupby(['NUM_POSTE']).sum()['RR'].max()
fig.update_yaxes(secondary_y= False, range= [0, max_value])  # Axe principale: Replace max_value with the desired maximum coordinate
fig.update_yaxes(secondary_y= True, title_text= "Cumulé", range= [0, max_value_cum]) # Axe secondaire: Replace max_value with the desired maximum coordinate

# Graduations de l'axe des DATES ...............................................................
if end_date - start_date > datetime.timedelta(days= 365): # Plus d'un an
    tickformat= '%b\n%Y'
    dtick= 'M6'
    showgrid_x= True
else: # Moins d'un an
    tickformat= '%b %Y'
    dtick= 'M1'
    showgrid_x= False
# ...............................Premiers subplots
# ................................................ en fonction de la longueur de la période
for i in range(1, len(postes) + 1):
    fig.update_xaxes(tickformat= tickformat, tickmode= 'linear', dtick= dtick, row=i, col=1, matches='x')
# ...............................Dernier subplot
# ................................................ en fonction de la longueur de la période
fig.update_xaxes(tickformat= tickformat, tickmode= 'linear', dtick= dtick, row=len(postes), col=1, matches='x')
#                                                SANS tenir compte de la longueur de la période
# fig.update_xaxes(tickformat= '%b\n%Y', tickmode= 'linear', dtick= 'M1', row=len(postes), col=1, matches='x')

# GRILLE .............................................................................
# .................. verticales...
fig.update_xaxes(showgrid= showgrid_x, gridwidth= 1, gridcolor= 'white')
# .................. horizontales
#                                Axe secondaire:
fig.update_yaxes(secondary_y=True, showgrid=False) # pas de grille

#-------------- sauvegarde le graphique
file_graph= "meteo.data MENS"
fig.write_html(os.path.join(folder_csv, file_graph + template_end[:-4] + ".html"))
# sauvegarde le graphique dans un fichier png
fig.write_image(os.path.join(folder_csv, file_graph + template_end[:-4] + ".png"))

#-------------- Valeurs quotidiennes pivotée pour la présentation avec les postes en colonnes
#               pour les postes et les paramètres SELECTIONNES, SUR la durée COMPLETE
for i, par in enumerate(param_list):
    if i==0:
        df_m = df[df['NUM_POSTE'].isin(postes)].pivot(index='DATE', columns='NOM_USUEL', values= param_list[i]) \
            .assign(parametre= par)
    else:
        # append des lignes à df_m
        df_m = pd.concat([df_m,
                        df[df['NUM_POSTE'].isin(postes)].pivot(index='DATE', columns='NOM_USUEL', values= param_list[i]) \
                        .assign(parametre= par)]
                        )
df_m= df_m.reset_index().set_index(['parametre', 'DATE'], drop= True)

#-------------- Affichage
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M"))
# imprimer un avertissement si la dernière date du dataframe est antérieure au dernier jour du mois en cours

titre= 'Données mensuelles des 2 derniers mois pour les paramètres choisis: '
titre= titre + 'Attention le cumul du dernier mois est incomplet à ce jour'
print(titre)
for i, par in enumerate(param_list):
    titre= '- ' + par + ': ' + df_desc_m.loc[par]['name_long']
    print(titre)
month_before_now= datetime.datetime(now.year, now.month, 1) - datetime.timedelta(days= 30)
display(df_m[(df_m.index.get_level_values(1) >= month_before_now) & (df_m.index.get_level_values(1) <= end_date)])

fig.show()

titre_m= "Données mensuelles LATEST (depuis le début de l'année précédente) pour les paramètres choisis: "
print(titre_m)
display(df_m)

2023-12-25 19:52
Données mensuelles des 2 derniers mois pour les paramètres choisis: Attention le cumul du dernier mois est incomplet à ce jour
- RR: cumul mensuel des hauteurs de précipitation (en mm et 1/10)
- TM: moyenne mensuelle des (TN+TX)/2 quotidiennes (en °C et 1/10)
- FXYAB: maximum absolu mensuel de la force maximale quotidienne du vent moyenné sur 10 mn (FXY), à 10 m (en m/s et 1/10)


NOM_USUEL             AIX EN PROVENCE  APT-VITON  CASTELLANE  FORCALQUIER  \
parametre DATE                                                              
RR        2023-11-01             30.2       37.6       102.2         63.1   
          2023-12-01             42.6       60.4        76.3          NaN   
TM        2023-11-01             10.2        9.1         7.1          NaN   
          2023-12-01              7.9        6.8         4.0          NaN   
FXYAB     2023-11-01              9.2        NaN         NaN          NaN   
          2023-12-01              7.8        NaN         NaN          NaN   

NOM_USUEL             FREJUS  LA BASTIDE DES JOURDANS  LE LUC  MARSEILLE-OBS  \
parametre DATE                                                                 
RR        2023-11-01    20.4                     22.0    21.7           23.2   
          2023-12-01    50.6                     42.6    53.8           28.3   
TM        2023-11-01    13.3                      9.0    12.7           12.9   
          2023-12-01    10.5                      7.0     9.2           10.5   
FXYAB     2023-11-01    14.2                     15.0    19.5            NaN   
          2023-12-01    15.1                     10.0    19.9            NaN   

NOM_USUEL             SALON DE PROVENCE  TOULON  VALENSOLE  VAUVENARGUES  
parametre DATE                                                            
RR        2023-11-01               30.8    25.8       36.4          41.8  
          2023-12-01               35.4    65.2       56.8          71.0  
TM        2023-11-01               10.5    13.6        9.0           8.9  
          2023-12-01                7.5    10.7        6.8           6.6  
FXYAB     2023-11-01               19.0    10.6       11.6          10.0  
          2023-12-01               18.4    12.1       11.2           8.3

Données mensuelles LATEST (depuis le début de l'année précédente) pour les paramètres choisis: 


NOM_USUEL             AIX EN PROVENCE  APT-VITON  CASTELLANE  FORCALQUIER  \
parametre DATE                                                              
RR        2022-01-01              3.6        1.8         5.6          3.5   
          2022-02-01             26.1       23.7        35.4         24.8   
          2022-03-01              6.6       11.6        13.4          6.6   
          2022-04-01             27.0       53.7        65.6         44.4   
          2022-05-01             26.0       47.3        29.4         28.5   
...                               ...        ...         ...          ...   
FXYAB     2023-08-01             11.3        NaN         NaN          NaN   
          2023-09-01              8.5        NaN         NaN          NaN   
          2023-10-01              9.4        NaN         NaN          NaN   
          2023-11-01              9.2        NaN         NaN          NaN   
          2023-12-01              7.8        NaN         NaN          NaN   

NOM_USUEL             FREJUS  LA BASTIDE DES JOURDANS  LE LUC  MARSEILLE-OBS  \
parametre DATE                                                                 
RR        2022-01-01     2.4                      3.4     0.8            0.4   
          2022-02-01    36.2                     18.4    31.9           19.9   
          2022-03-01    30.4                      6.4    28.2            9.4   
          2022-04-01    40.4                     41.5    45.8           19.3   
          2022-05-01     3.8                     24.5    38.1           29.5   
...                      ...                      ...     ...            ...   
FXYAB     2023-08-01    13.1                     13.3    17.1            NaN   
          2023-09-01    10.7                      9.2    11.8            NaN   
          2023-10-01    11.3                     10.2    12.1            NaN   
          2023-11-01    14.2                     15.0    19.5            NaN   
          2023-12-01    15.1                     10.0    19.9            NaN   

NOM_USUEL             SALON DE PROVENCE  TOULON  VALENSOLE  VAUVENARGUES  
parametre DATE                                                            
RR        2022-01-01                2.0     2.8        4.8           1.0  
          2022-02-01               38.2    27.0       15.0          24.7  
          2022-03-01                6.6    16.4        8.4          10.4  
          2022-04-01               25.6    26.2       32.6          38.9  
          2022-05-01                7.6    28.6       29.0          44.7  
...                                 ...     ...        ...           ...  
FXYAB     2023-08-01               16.6    11.0       13.9           8.3  
          2023-09-01               11.8     9.2       10.1           6.7  
          2023-10-01               12.0    11.9       10.7           8.3  
          2023-11-01               19.0    10.6       11.6          10.0  
          2023-12-01               18.4    12.1       11.2           8.3  

[72 rows x 12 columns]

##### 3) Sauvegarde les données intégrales dans un fichier Excel (tous les paramètres pour tous les postes des départements concernés)
Le fichier est sauvegardé dans le même dossier local que les fichiers CSV décompressés

In [28]:
# Sauvegarde toutes les informations dans un fichier Excel
print(now.strftime("%Y-%m-%d %H:%M"))
print('Sauvegarde en cours, patientez...')
file= os.path.join(folder_csv, "meteo.data MENS" + '_' + '-'.join(departements) + template_end[:-4] + ".xlsx")
writer = pd.ExcelWriter(file, engine='xlsxwriter')
workbook  = writer.book
#======================== Sauvegarde la liste des postes dans le fichier excel
# Lecture en ligne du fichier JSON
req= requests.get(url_liste_postes)
if req.status_code == 200:
    data_json= req.json()
else:
    print('la requête a échoué avec le code : ', req.status_codes)
    sys.exit() # interrompt le script
df_liste_postes = pd.DataFrame(data_json['features']).T
# Ajout de colonnes avec les champs de properties
df_liste_postes['lat'] = df_liste_postes['geometry'].apply(lambda x: x['coordinates'][1]).astype(float)
df_liste_postes['lon'] = df_liste_postes['geometry'].apply(lambda x: x['coordinates'][0]).astype(float)
df_liste_postes['nom_usuel']= df_liste_postes['properties'].apply(lambda x: x['NOM_USUEL'].strip())
df_liste_postes['num_poste']= df_liste_postes['properties'].apply(lambda x: x['NUM_POSTE'].strip())
df_liste_postes['commune'] = df_liste_postes['properties'].apply(lambda x: x['COMMUNE'].strip())
df_liste_postes['ficheClimComplete'] = df_liste_postes['properties'].apply(lambda x: x['ficheClimComplete']).astype(float)
df_liste_postes['ficheClimReduite'] = df_liste_postes['properties'].apply(lambda x: x['ficheClimReduite']).astype(float)
df_liste_postes['alti'] = df_liste_postes['properties'].apply(lambda x: x['ALTI'])
# sauvegarde dans un fichier excel plus lisible que le JSON après suppression des colonnes type, geometry et properties
df_liste_postes.drop(['type', 'geometry', 'properties'], axis=1, inplace=True)
df_liste_postes.sort_values(by= ['num_poste'], inplace=True)
sheet= 'liste_postes'
df_liste_postes.to_excel(writer, sheet_name= sheet, index= False, startrow= 0)

#======================== Sauvegarde la liste des champs dans le fichier Excel
sheet= 'MENSQ_descriptif_champs'
df_desc_m.to_excel(writer, sheet_name= sheet, index= True, startrow= 1)
worksheet = writer.sheets[sheet]
worksheet.write('A1', sheet + '.csv')
                
sheet= 'donnée complètes'
df.to_excel(writer, sheet_name= sheet, index=False, startrow= 2)
worksheet = writer.sheets[sheet]
worksheet.write('A1', now.strftime("%Y-%m-%d %H:%M"))
worksheet.write('A2', 'Données LATEST quotidiennes au format original, complètes pour tous les départements concernés: ')

#======================== Sauvegarde les dataframes dans le fichier Excel
sheet= 'donnée complètes'
df.to_excel(writer, sheet_name= sheet, index=False, startrow= 2)
worksheet = writer.sheets[sheet]
worksheet.write('A1', now.strftime("%Y-%m-%d %H:%M"))
worksheet.write('A2', "Données mensuelles LATEST (depuis le début de l'année précédente) au format original, complètes pour tous les départements concernés: ")

# ajoute un onglet exel avec df_m
n_param= len(param_list)
sheet= 'MENS Sélection'
df_m.to_excel(writer, sheet_name= sheet, index=True, startrow= n_param + 2)
worksheet = writer.sheets[sheet]
worksheet.write('A1', now.strftime("%Y-%m-%d %H:%M"))
worksheet.write('A2', titre_m)
for i, par in enumerate(param_list):
    titre= '- ' + par + ': ' + df_desc_m.loc[par]['name_long']
    worksheet.write('A' + str(i+3), titre)

# ajoute un onglet excel et enregistre une image statique du graphique
sheet= 'graph'
worksheet = workbook.add_worksheet(sheet)
worksheet.insert_image('A1', os.path.join(folder_csv, file_graph + template_end[:-4] + ".png"))

writer.save()

print(now.strftime("%Y-%m-%d %H:%M"))
print('Ficher Excel sauvegardé: ', file)

2023-12-25 19:51
Sauvegarde en cours, patientez...
2023-12-25 19:51
Ficher Excel sauvegardé:  X:\1-COMMUN\DIS\Documentation\Hydrologie\Documentation externe\Climat France\Météo-France\meteo.data\base\M\meteo.data MENS_04-13-83-84_latest-2022-2023.xlsx
